In [13]:
import os 
import sys 
from SourceFiles.teproteus import TEProteusAdmin as TepAdmin
import matplotlib.pyplot as plt
import numpy as np

In [14]:
# connect and initialize
admin = TepAdmin() #required to control PXI module
sid = 8 #PXI slot WDS found
inst = admin.open_instrument(slot_id=sid)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp) # Print *IDN
inst.send_scpi_cmd('*CLS; *RST')

Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
connected to: slot#: 8,


0

In [15]:

sampleRateDAC = 8E9
inst.send_scpi_cmd(':FREQ:RAST {0}'.format(2.5E9))
inst.send_scpi_cmd(':INST:CHAN {0}'.format(1))
inst.send_scpi_cmd(':MODE DUC')
inst.send_scpi_cmd(':SOUR:INT X8')
inst.send_scpi_cmd(':FREQ:RAST {0}'.format(sampleRateDAC))
inst.send_scpi_cmd(':IQM ONE')

inst.send_scpi_cmd(':SOUR:NCO:CFR1 10E6')

inst.send_scpi_cmd(':FREQ:RAST {0}'.format(2.5E9))
inst.send_scpi_cmd(':INST:CHAN {0}'.format(2))
inst.send_scpi_cmd(':MODE DUC')
inst.send_scpi_cmd(':SOUR:INT X8')
inst.send_scpi_cmd(':FREQ:RAST {0}'.format(sampleRateDAC))
inst.send_scpi_cmd(':IQM ONE')
inst.send_scpi_cmd(':SOUR:NCO:CFR1 10E6')



inst.send_scpi_query(':INT?')

inst.send_scpi_query(':INT?')

'X8'

In [16]:
resp=inst.send_scpi_query(':SYST:ERR?')

print(resp)

0, no error


In [17]:
inst.send_scpi_query(':FREQ:RAST?')

'8000000000.000'

In [18]:

max_dac=65535 # Max Dac
half_dac=max_dac/2 # DC Lev
data_type = np.uint16 # DAC data type

segTime =16000

 

dacWaveOn_I = np.ones(segTime)

dacWaveOn_I = dacWaveOn_I * max_dac

dacWaveOn_I = dacWaveOn_I.astype(data_type)

dacWaveOn_Q = np.ones(segTime)*(32768)

dacWaveOn_Q = dacWaveOn_Q.astype(data_type)

# reshapes for IQIQIQIQIQ....


arr_tuple = (dacWaveOn_I, dacWaveOn_Q)

dacWaveOn_IQ = np.vstack(arr_tuple).reshape((-1,), order='F')

print(len(dacWaveOn_IQ))


32000


In [19]:
dacWaveOn_IQ

array([65535, 32768, 65535, ..., 32768, 65535, 32768], dtype=uint16)

In [20]:
# select CH1
ch = 1 # everything after relates to CH 1 
segnum = 1
inst.send_scpi_cmd('*CLS')
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TRAC:DEF {0}, {1}'.format(segnum, len(dacWaveOn_IQ)))
inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))
inst.write_binary_data(':TRAC:DATA', dacWaveOn_IQ) # write, and wait while *OPC completes

inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')

inst.send_scpi_cmd(':TASK:COMP:ENAB INT')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')


inst.send_scpi_cmd(':OUTP ON')
inst.send_scpi_query(':SYST:ERR?')

'0, no error'

In [21]:
ch = 2 # everything after relates to CH 2
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')

inst.send_scpi_cmd(':TASK:COMP:ENAB INT')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

0

In [22]:
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRIG:COUP ON')
inst.send_scpi_cmd('*TRG')


# inst.send_scpi_cmd(':TRIG:SEL TRG1')
# inst.send_scpi_cmd(':TRIG:STAT ON')
# inst.send_scpi_cmd(':TRIG:LEV 1.0')









0